In [1]:
import torch
from torch.utils.data import Dataset
from torch.utils.data import DataLoader
import os
import math
import time

In [2]:
torch.cuda.is_available()
torch.cuda.FloatTensor()

tensor([], device='cuda:0')

In [3]:
# define dataset
class BowlingDataset(Dataset):
    def __init__(self, fileName):
        self.f = open(fileName, "rb")
        self.length = int(os.stat(fileName).st_size/27)
        
    def __len__(self):
        return self.length
    
    def __getitem__(self, idx):
        self.f.seek(idx)
        gameData = self.f.read(27)
        tempArray = []
        for x in range(25):
            tempArray += [v for v in format(gameData[x], "08b")]
        finalScore = gameData[-1] * 256 + gameData[-2]
        inputArray = [float(v) for v in tempArray][:120]
        output = finalScore
        return torch.tensor(inputArray), torch.tensor(float(output))

In [4]:
class TestModel(torch.nn.Module):
    def __init__(self):
        super().__init__()
        self.relu_stack = torch.nn.Sequential(
            torch.nn.Linear(120, 512),
            torch.nn.ReLU(),
            torch.nn.Linear(512, 256),
            torch.nn.ReLU(),
            torch.nn.Linear(256, 64),
            torch.nn.ReLU(),
            torch.nn.Linear(64, 1),
        )
        
    def forward(self, x):
        logits = self.relu_stack(x)
        output = torch.nn.Sigmoid()(logits)
        return output * 300

In [5]:
model(trainData[0][0].cuda()).is_cuda

NameError: name 'model' is not defined

In [ ]:
trainData[0][0].is_pinned()

In [6]:
def train_loop(dataloader, model, loss_fn, optimizer):
    size = len(dataloader.dataset) # get number of samples
    totalBatches = len(dataloader)
    model.train() # need to look into what this exactly does
    startTime = time.time()
    for batchNum, (x, y) in enumerate(dataloader):
        # zero out gradients
        optimizer.zero_grad()
        
        # prediction + loss
        prediction = model(x.to("cuda")).squeeze(1)
        loss = loss_fn(prediction, y.to("cuda"))
        
        #Backpropagation
        loss.backward()
        optimizer.step()
        
        if not batchNum % 1000:
            print(f"loss: {loss.item():.2f}\tbatch num: {batchNum}/{totalBatches}")
            print(f"took {time.time() - startTime} seconds")
            startTime = time.time()

In [7]:
def test_loop(dataloader, model):
    model.eval() # need to look into what this does
    size = len(dataloader.dataset)
    numBatches = len(dataloader)
    test_loss = 0
    
    with torch.no_grad():
        for x, y in dataloader:
            pred = model(x)
            test_loss += loss_fn(pred, y).item()
    
    print(f"Average Loss: {test_loss/numBatches}")

In [8]:
# HYPERPARAMS
batch_size = 64
learning_rate = 0.001
epochs = 8

In [9]:
trainData = BowlingDataset("ScoreDetailDataset.txt")
trainDataLoader = DataLoader(trainData, batch_size=batch_size, shuffle=True, pin_memory=True) #pin memory doesnt do shit bc the memory has to be grabbed from a physical file
testData = BowlingDataset("ScoreDetailDatasetVSplit.txt")
testDataLoader = DataLoader(testData, batch_size=batch_size, shuffle=True)

In [10]:
# test to see if pinning is working
for batch_ndx, sample in enumerate(trainDataLoader):
    if batch_ndx > 5:
        break
    print(sample[1].is_pinned())
    print(sample[1].is_cuda)


True
False
True
False
True
False
True
False
True
False
True
False


In [11]:
model = TestModel().cuda()
loss_fn = torch.nn.L1Loss().cuda()
optimizer = torch.optim.Adam(model.parameters(), lr=learning_rate)

for t in range(epochs):
    startTime = time.time()
    print(f"Starting epoch {t}")
    train_loop(trainDataLoader, model, loss_fn, optimizer)
    print(f"Epoch {t} took {time.time() - startTime} seconds")
    test_loop(testDataLoader, model)
print("Finished")

Starting epoch 0
loss: 15764.66	batch num: 0/129769
took 2.497081756591797 seconds
loss: 16446.31	batch num: 1000/129769
took 8.260373830795288 seconds
loss: 15691.47	batch num: 2000/129769
took 8.54046106338501 seconds
loss: 14944.53	batch num: 3000/129769
took 8.604514598846436 seconds
loss: 17730.17	batch num: 4000/129769
took 8.613401651382446 seconds
loss: 18323.27	batch num: 5000/129769
took 8.67867922782898 seconds
loss: 18958.84	batch num: 6000/129769
took 8.497145175933838 seconds
loss: 20722.02	batch num: 7000/129769
took 8.641990661621094 seconds
loss: 26092.32	batch num: 8000/129769
took 8.551717519760132 seconds
loss: 13864.75	batch num: 9000/129769
took 8.454698085784912 seconds
loss: 15867.88	batch num: 10000/129769
took 8.594767808914185 seconds
loss: 17539.86	batch num: 11000/129769
took 8.590681552886963 seconds
loss: 18509.47	batch num: 12000/129769
took 8.558688879013062 seconds
loss: 19870.05	batch num: 13000/129769
took 8.450331449508667 seconds
loss: 16818.45	bat

KeyboardInterrupt: 